In [185]:
import pandas as pd
import glob
import re
import collections
import numpy as np
import matplotlib.pyplot as plt
import gzip
import datetime
import pickle
import itertools

from tqdm import tqdm_notebook
from nltk.util import ngrams
from wordcloud import WordCloud
from IPython.display import display, HTML

import spacy
from spacy import displacy
from collections import Counter
from bisect import bisect_left

nlp = spacy.load('en_core_web_sm')

In [2]:
# Debug
print(datetime.datetime.now().time())

22:29:13.013116


# Create featureset
Load the dataset with the generated synthetic candidate queries

In [323]:
syn_candidate_queries.to_pickle('/Users/rwkoops/PycharmProjects/IR_project/IR_project_02/created_sample/syn_candidate_queries_5m.pickle')

In [324]:
syn_candidate_queries = pd.read_pickle('/Users/rwkoops/PycharmProjects/IR_project/IR_project_02/created_sample/syn_candidate_queries_5m.pickle')

# Create N-gram & NER features

#### Load it

In [10]:
with open('/Users/rwkoops/PycharmProjects/IR_project/IR_project_02/created_sample/hist_suffixes.pickle_1m', 'rb') as f:
    suffixes = pickle.load(f)

#### Create N-gram features based on the "Query_clean"-column

In [11]:
historical_dict = collections.Counter(suffixes)

def ngram_freq_per_n(candidate, historical_dict, n):
    words = candidate.split()
    ngram_n = 0
    ngrams_i = ngrams(words, n)
    
    for word in ngrams_i:
        freq_g = historical_dict[" ".join(word)]
        ngram_n += freq_g
        
    return ngram_n

In [14]:
ngram_max = 6

for i in range(1, ngram_max + 1):
    ngram_name = 'ngram_' + str(i)
    syn_candidate_queries[ngram_name] = syn_candidate_queries[syn_candidate_queries.notnull()]['Query_clean'].map(lambda candidate_row: ngram_freq_per_n(candidate_row, historical_dict, i))

#### Create NER Features based on the "Synthetic_query"-column

In [15]:
def ner_contains_and_count_norm(candidate): 
    entities = nlp(str(candidate))
    contains = 0
    ner_norm = 0
    
    if len(entities) > 0:
        contains = 1
        ner_norm = len(entities) / len(candidate)
    
    return [contains, ner_norm]

syn_candidate_queries['has_ne'], syn_candidate_queries['ne_norm'] = zip(*syn_candidate_queries[syn_candidate_queries.notnull()]['Synthetic_query'].map(lambda candidate_row: ner_contains_and_count_norm(candidate_row)))

# Create 'Other'-features

In [ ]:
def get_other_features(prefix, suffix, candidate, historical_logs):
    # Boolean indicating whether the prefix ends with a space
    bool_space = prefix.endswith(" ")
    
    # The frequency of the candidate in the historical logs
    frequency = historical_logs[candidate]
    
    # Prefix, suffix and total length in characters
    prefixlen_char = len(prefix)
    suffixlen_char = len(suffix)
    totallen_char = len(candidate)
    
    # Prefix, suffix and total length in words
    prefixlen_word = len(prefix.split())
    suffixlen_word = len(suffix.split())
    totallen_word = len(candidate.split())
    
    return [frequency, 
            prefixlen_char, suffixlen_char, totallen_char,
            prefixlen_word, suffixlen_word, totallen_word,
            bool_space * 1]

In [ ]:
syn_candidate_queries['candid_freq'], syn_candidate_queries['prefixlen_char'], syn_candidate_queries['suffixlen_char'], syn_candidate_queries['totallen_char'], syn_candidate_queries['prefixlen_word'], syn_candidate_queries['suffixlen_word'], syn_candidate_queries['totallen_word'] = zip(*queries.apply(lambda query_row: get_other_features(query_row.Prefix, query_row.Suffix, query_row.Synthetic_query, historical_dict), axis=1))

In [ ]:
# Debug
print(datetime.datetime.now().time())